QC of ETL starting with GDC release 22 for programs TARGET, ORGANOID, and BEATAML. 


This notebook focuses on the QC of program BEATAML data_type RNA-Seq

##QC table checklist 

**1. Check schema**

Are all the fields labeled?

Is there a table description?

Do the field labels make sense for all fields?
    
Are the labels correct?

**2. Look at table row number and size**

Do these metrics make sense?

**3. Scroll through table manually**

See if anything stands out - empty columns, etc.

The BigQuery table search user interface is useful in for this test run. The test tier points to the isb-etl-open. 

[ISB-CGC BigQuery table  search test tier](https://isb-cgc-test.appspot.com/bq_meta_search/)

Run a manual check in the console with the steps mentioned in step 1.

**4. Number of cases on GDC portal versus table?**

**5. Number of cases / aliquots versus BigQuery metadata table**

**6. Number of entries per gene - should equal aliquot count**

**7.Check for any duplicate rows present in the table**

##Reference material



*   [NextGenETL](https://github.com/isb-cgc/NextGenETL) GitHub repository
*   [ETL QC SOP draft](https://docs.google.com/document/d/1Wskf3BxJLkMjhIXD62B6_TG9h5KRcSp8jSAGqcCP1lQ/edit)

##Before you begin

You need to load the BigQuery module, authenticate ourselves, create a client variable, and load the necessary libraries.


In [0]:
from google.colab import auth
try:
  auth.authenticate_user()
  print('You have been successfully authenticated!')
except:
  print('You have not been authenticated.')

Authorized


In [0]:
from google.cloud import bigquery
try:
  project_id = 'isb-project-zero' # Update your_project_number with your project number
  client = bigquery.Client(project=project_id)
  print('BigQuery client successfully initialized')
except:
  print('Failed')

BigQuery client successfully initialized


In [0]:
#Install pypika to build a Query 
!pip install pypika
# Import from PyPika
from pypika import Query, Table, Field, Order

import pandas

     |████████████████████████████████| 61kB 2.0MB/s 
  Created wheel for pypika: filename=PyPika-0.37.6-py2.py3-none-any.whl size=42748 sha256=5de225654c66c5f1acaaca73ecadf0d4fd8e55e21a9d2ff54b912a2f54b474a5
  Stored in directory: /root/.cache/pip/wheels/7e/39/df/d08ca9b40bba9f6d626a32c2e49c1ba61441eaa166f2cc8eb5
Successfully built pypika


## READY TO BEGIN TESTING

##Program BEATAML

**Testing Full ID** `isb-project-zero.RNAseq_Gene_Expression.BEATAML1_0_RNAseq_Gene_Expression`

[Table location](https://console.cloud.google.com/bigquery?authuser=2&project=isb-project-zero&p=isb-project-zero&d=RNAseq_Gene_Expression&t=BEATAML1_0_RNAseq_Gene_Expression&page=table)

Source : GDC API

Date Created : 	Apr 1, 2020, 7:06:22 PM

Release version : v22


##test 1 - schema verification

**1. Check schema**

Are all the fields labeled?

Is there a table description?

Do the field labels make sense for all fields
    
Are the labels correct

Google documentation column descriptions for [reference](https://cloud.google.com/bigquery/docs/information-schema-tables#column_field_paths_view).

Google documentation table options for [reference](https://cloud.google.com/bigquery/docs/information-schema-tables#options_table).

In [0]:
#return all table information for dataset RNAseq_Gene_Expression 
rnaseq_table = Table('`isb-project-zero`.RNAseq_Gene_Expression.INFORMATION_SCHEMA.TABLES')
rnaseq_query = Query.from_(rnaseq_table) \
                  .select(' table_catalog, table_schema, table_name, table_type ') \
                  .where(rnaseq_table.table_name=='BEATAML1.0_RNAseq_Gene_Expression') \
                  
rnaseq_query_clean = str(rnaseq_query).replace('"', "")
rnaseq = client.query(rnaseq_query_clean).to_dataframe()
rnaseq.head()

,table_catalog,table_schema,table_name,table_type


In [0]:
#return all table information for dataset RNAseq_Gene_Expression 
rnaseq_table = Table('`isb-project-zero`.RNAseq_Gene_Expression.INFORMATION_SCHEMA.TABLE_OPTIONS')
rnaseq_query = Query.from_(rnaseq_table) \
                  .select(' table_name, option_name, option_type, option_value ') \
                  .where(rnaseq_table.table_name=='BEATAML1_0_RNAseq_Gene_Expression') \

rnaseq_query_clean = str(rnaseq_query).replace('"', "")
rnaseq = client.query(rnaseq_query_clean).to_dataframe()
pandas.options.display.max_rows
#rnaseq

for i in range(len(rnaseq)):
    print(rnaseq['option_name'][i] + '\n')
    print('\t' + rnaseq['option_value'][i] + '\n')
    print('\t' + rnaseq['option_type'][i] + '\n')

friendly_name

	"BEATAML1.0 RNASEQ GENE EXPRESSION"

	STRING

description

	"Data was extracted from GDC on March 2020. mRNA expression data was generated using Illumina GA or HiSeq sequencing platforms with information from each of the three files (HTSeq Counts, HTSeq FPKM, HTSeq FPKM-UQ) from the GDC's RNAseq pipeline was combine for each aliquot."

	STRING

labels

	[STRUCT("reference_genome_0", "hg38"), STRUCT("data_type", "gene_expression"), STRUCT("access", "open"), STRUCT("category", "processed_-omics_data"), STRUCT("status", "current"), STRUCT("source", "gdc"), STRUCT("program", "beataml"), STRUCT("experimental_strategy", "rnaseq")]

	ARRAY<STRUCT<STRING, STRING>>



In [0]:
#check for empty schemas in dataset RNAseq_Gene_Expression 
rnaseq_table = Table('`isb-project-zero`.RNAseq_Gene_Expression.INFORMATION_SCHEMA.TABLE_OPTIONS')
rnaseq_query = Query.from_(rnaseq_table) \
                  .select(' table_name, option_name, option_type, option_value ') \
                  .where(rnaseq_table.table_name=='BEATAML1_0_RNAseq_Gene_Expression') \

rnaseq_query_clean = str(rnaseq_query).replace('"', "")
rnaseq = client.query(rnaseq_query_clean).to_dataframe()
pandas.options.display.max_rows
print("Are there any empty cells in the table schema?")
pandas.isnull(rnaseq).values.any()

Are there any empty cells in the table schema?


False

FIELD Descriptions pulled example below


In [0]:
#list of field descriptions for table 

#return all table information for dataset RNAseq_Gene_Expression 
rnaseq_table = Table('`isb-project-zero`.RNAseq_Gene_Expression.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS')
rnaseq_query = Query.from_(rnaseq_table) \
                  .select('table_name, column_name, description') \
                  .where(rnaseq_table.table_name=='BEATAML1_0_RNAseq_Gene_Expression') \

rnaseq_query_clean = str(rnaseq_query).replace('"', "")
rnaseq = client.query(rnaseq_query_clean).to_dataframe()
pandas.options.display.max_rows
#rnaseq
for i in range(len(rnaseq)):
  print(rnaseq['column_name'][i] + '\n')
  print('\t' + rnaseq['description'][i] + '\n')

project_short_name

	Project name abbreviation; the program name appended with a project name abbreviation; eg. TCGA-OV, etc.

case_barcode

	Original case barcode

sample_barcode

	sample barcode, eg TCGA-12-1089-01A. One sample may have multiple sets of CN segmentations corresponding to multiple aliquots; use GROUP BY appropriately in queries

aliquot_barcode

	TCGA aliquot barcode, eg TCGA-12-1089-01A-01D-0517-31

gene_name

	Gene name e.g. TTN, DDR1, etc.

gene_type

	The type of genetic element the reads mapped to, eg protein_coding, ribozyme

Ensembl_gene_id

	The Ensembl gene ID from the underlying file, but stripped of the version suffix -- eg ENSG00000185028

Ensembl_gene_id_v

	The Ensembl gene ID from the underlying file, including the version suffix  --  eg ENSG00000235943.1

HTSeq__Counts

	Number of mapped reads to each gene as calculated by the Python package HTSeq. https://docs.gdc.cancer.gov/Encyclopedia/pages/HTSeq-Counts/

HTSeq__FPKM

	FPKM is implemented at the GDC

In [0]:
#list of field descriptions for table 

#check for empty schemas in dataset RNAseq_Gene_Expression 
rnaseq_table = Table('`isb-project-zero`.RNAseq_Gene_Expression.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS')
rnaseq_query = Query.from_(rnaseq_table) \
                  .select('table_name, column_name, description') \
                  .where(rnaseq_table.table_name=='BEATAML1_0_RNAseq_Gene_Expression') \

rnaseq_query_clean = str(rnaseq_query).replace('"', "")
rnaseq = client.query(rnaseq_query_clean).to_dataframe()
pandas.options.display.max_rows
print("Are there any empty cells in the table schema?")
pandas.isnull(rnaseq).values.any()

Are there any empty cells in the table schema?


False

##test 2 - row number verification

**2. Look at table row number and size**

Do these metrics make sense?

In [0]:
%%bigquery --project isb-project-zero
SELECT COUNT(project_short_name)
FROM `isb-project-zero.RNAseq_Gene_Expression.BEATAML1_0_RNAseq_Gene_Expression`

,f0_
0,30846330


##test 3 - manual verification

**3. Scroll through table manually**

See if anything stands out - empty columns, etc.

The BigQuery table search user interface is useful in for this test run. The test tier points to the isb-etl-open. 

ISB-CGC BigQuery table  search [test tier](https://isb-cgc-test.appspot.com/bq_meta_search/).

BigQuery console [isb-project-zero](https://console.cloud.google.com/bigquery?authuser=1&folder=&organizationId=&project=isb-project-zero&p=isb-project-zero&d=RNAseq_Gene_Expression&t=BEATAML1_0_RNAseq_Gene_Expression&page=table).

Run a manual check in the console with the steps mentioned in step 1 

Are all the fields labeled?

Is there a table description?

Do the field labels make sense for all fields?
    
Are the labels correct?

##test 4 - GDC Data Portal count verfication


**4. Number of cases on GDC portal versus table?**

In [0]:
# Query below will display the number of cases presents in this table.

rnaseq_table = Table('`isb-project-zero.RNAseq_Gene_Expression.BEATAML1_0_RNAseq_Gene_Expression`')
rnaseq_query = Query.from_(rnaseq_table) \
                  .select(' DISTINCT case_barcode, count(*) as count') \
                  .groupby('case_barcode')

rnaseq_query_clean = str(rnaseq_query).replace('"', "")
#print(rnaseq_query_clean)
rnaseq = client.query(rnaseq_query_clean).to_dataframe()
print('number of cases = ' + str(len(rnaseq.index)))

# rnaseq.set_option("display.max_rows", None, "display.max_columns", None)

number of cases = 450


To copmare against the GDC Data Portal, 
you first go the GDC Data Portal and search for program BEATAML1.0-COHORT and experimental_strategy RNA-Seq, the cases number returned is 450. 

[GDC Data portal](https://portal.gdc.cancer.gov/repository?facetTab=cases&filters=%7B%22op%22%3A%22and%22%2C%22content%22%3A%5B%7B%22op%22%3A%22in%22%2C%22content%22%3A%7B%22field%22%3A%22cases.project.project_id%22%2C%22value%22%3A%5B%22BEATAML1.0-COHORT%22%5D%7D%7D%2C%7B%22op%22%3A%22in%22%2C%22content%22%3A%7B%22field%22%3A%22files.experimental_strategy%22%2C%22value%22%3A%5B%22RNA-Seq%22%5D%7D%7D%5D%7D&searchTableTab=cases) filter results. 

##test 5 - file metadata table count verification

**5. Number of cases / aliquots versus BigQuery metadata table**

RNA-Seq cases counts table results below.

In [0]:
# Query below will display the number of cases presents in this table.

rnaseq_table = Table('`isb-project-zero.RNAseq_Gene_Expression.BEATAML1_0_RNAseq_Gene_Expression`')
rnaseq_query = Query.from_(rnaseq_table) \
                  .select(' DISTINCT case_barcode, count(*) as count') \
                  .groupby('case_barcode')

rnaseq_query_clean = str(rnaseq_query).replace('"', "")
#print(rnaseq_query_clean)
rnaseq = client.query(rnaseq_query_clean).to_dataframe()
print('number of cases = ' + str(len(rnaseq.index)))

# rnaseq.set_option("display.max_rows", None, "display.max_columns", None)

number of cases = 450


GDC file metadata table cases count for RNA-seq below

In [0]:
%%bigquery --project isb-project-zero
SELECT case_gdc_id, program_name
FROM `isb-project-zero.GDC_metadata.rel23_fileData_current`
where program_name = 'BEATAML1.0'
and experimental_strategy = 'RNA-Seq'
and analysis_workflow_type IN ('HTSeq - Counts', 'HTSeq - FPKM-UQ', 'HTSeq - FPKM')
group by case_gdc_id, program_name


,case_gdc_id,program_name
0,73a6c3ff-be0e-490d-a783-0b73db496e4e,BEATAML1.0
1,fe680216-74b5-4568-b546-79baa87de538,BEATAML1.0
2,62637181-db4b-4674-a779-c5c0a46a6733,BEATAML1.0
3,98c21e0b-1373-4785-8edf-345c55fd370c,BEATAML1.0
4,dd78709c-de82-49b8-a0f7-3b8df37337b0,BEATAML1.0
...,...,...
445,55a99310-cbf8-4c6e-bfcc-d70a5b538888,BEATAML1.0
446,406f1a09-be07-4a16-aa98-5922afc43262,BEATAML1.0
447,079c625e-2c17-471d-8822-57af5d191626,BEATAML1.0
448,5f27a823-9db2-410c-bf15-6ce186523e95,BEATAML1.0


RNA-Seq aliquot counts table results below.

In [0]:
# Query below will display the number of cases presents in this table.

rnaseq_table = Table('`isb-project-zero.RNAseq_Gene_Expression.BEATAML1_0_RNAseq_Gene_Expression`')
rnaseq_query = Query.from_(rnaseq_table) \
                  .select(' distinct aliquot_gdc_id, count(*) as count') \
                  .groupby('aliquot_gdc_id')

rnaseq_query_clean = str(rnaseq_query).replace('"', "")
#print(rnaseq_query_clean)
rnaseq = client.query(rnaseq_query_clean).to_dataframe()
print('number of aliquots = ' + str(len(rnaseq.index)))

# rnaseq.set_option("display.max_rows", None, "display.max_columns", None)

number of aliquots = 510


GDC file metadata table aliquot count for RNA-seq below.

In [0]:
%%bigquery --project isb-project-zero
select distinct associated_entities__entity_gdc_id 
from `isb-cgc.GDC_metadata.rel22_fileData_active` 
where program_name = "BEATAML1.0"
and experimental_strategy = "RNA-Seq"
and analysis_workflow_type IN ('HTSeq - Counts', 'HTSeq - FPKM-UQ', 'HTSeq - FPKM')

,associated_entities__entity_gdc_id
0,020c8100-4241-4a1e-9b83-e0c5bf9ca24c
1,02dd7a9d-278e-455e-859d-8c7879bd59f6
2,08deb765-5dac-4e88-8f14-2834398f4ab5
3,07bd3faf-23c9-474c-8d98-c85892e7bb2c
4,0268dd28-13db-492e-9cbe-77e79788a8ab
...,...
505,fee6fcd3-e2d0-471e-a08d-b13204831746
506,fe453a4b-4f57-4415-993a-25b5089a9eb4
507,ff021599-1c41-4a3d-a7ee-64a54e28c472
508,fec295bf-d2ae-4829-8c3e-67588fbbcfb8


## test 6 - gene entry verification

**6. Number of entries per gene - should equal aliquot count**

In [0]:
%%bigquery --project isb-project-zero

select distinct Ensembl_gene_id_v, count(Ensembl_gene_id_v) as count
from `isb-project-zero.RNAseq_Gene_Expression.BEATAML1_0_RNAseq_Gene_Expression` 
group by Ensembl_gene_id_v 
order by count

,Ensembl_gene_id_v,count
0,ENSG00000254017.1,510
1,ENSG00000251791.1,510
2,ENSG00000234911.1,510
3,ENSG00000211786.3,510
4,ENSG00000261026.1,510
...,...,...
60478,ENSG00000259918.1,510
60479,ENSG00000262583.1,510
60480,ENSG00000230583.5,510
60481,ENSG00000234429.1,510


##step 7 - duplication verifcation

**7.Check for any duplicate rows present in the table**

In [0]:
%%bigquery --project isb-project-zero

SELECT count(project_short_name) as count
FROM `isb-project-zero.RNAseq_Gene_Expression.BEATAML1_0_RNAseq_Gene_Expression` 
GROUP BY project_short_name, case_barcode, sample_barcode, aliquot_barcode, gene_name, gene_type, Ensembl_gene_id, Ensembl_gene_id_v, HTSeq__Counts, HTSeq__FPKM, HTSeq__FPKM_UQ, case_gdc_id, sample_gdc_id, aliquot_gdc_id, file_gdc_id_counts, file_gdc_id_fpkm, file_gdc_id_fpkm_uq, platform
order by count desc
limit 10

#query to be run manually for duplication verification QC

,count
0,1
1,1
2,1
3,1
4,1
...,...
30846325,1
30846326,1
30846327,1
30846328,1
